In [1]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
from rivnet import reproject_arr_to_match_profile, reproject_profile_to_new_crs
from pathlib import Path
import glob
from tqdm import tqdm
from rasterio.warp import calculate_default_transform

In [2]:
out = Path('out')
out.mkdir(exist_ok=True)

In [3]:
tiles = ['NH15', 'NH16', 'NR08']

In [4]:
if map_name == 'stamen_terrain_12':
    target_resolution = 0.0003
elif map_name == 'google_16':
    #  divide by 15 if want the 2 meter raster, but probably not necessary for viz
    target_resolution = 0.0003 #/ 15
else:
    raise ValueError('only works for google_16 and stamen_terrain_12')


src_arr_4326, src_profile_4326 = reproject_arr_to_new_crs(src_arr, 
                                                          src_profile, 
                                                          'epsg:4326',
                                                          resampling='nearest',
                                                          target_resolution=target_resolution
                                                          )

NameError: name 'map_name' is not defined

In [11]:
def get_reference_tif_profile(tile_name):
    
    tile_dir = out/tile_name
    with rasterio.open(tile_dir/'distance.tif') as ds:
        src_profile = ds.profile
    profile_4326 = reproject_profile_to_new_crs( src_profile, 
                                                'epsg:4326',
                                                 target_resolution= 0.0003
                                                )
    
    return profile_4326

def reproject_singe_band(tif_path, reference_profile, out_dir):
    with rasterio.open(tif_path) as ds:
        band = ds.read(1)
        profile = ds.profile
    band_r, _ = reproject_arr_to_match_profile(band, profile, reference_profile, resampling='nearest')
    band_r = band_r[0, ...]
    p = reference_profile.copy()
    dtype = profile['dtype']
    p['dtype'] = dtype
    with rasterio.open(out_dir/tif_path.name, 'w', **p) as ds:
        ds.write(band_r.astype(dtype), 1)
    return out_dir/tif_path.name

def reproject_tile_data(tile_name):    
    tile_dir = out/tile_name
    out_dir = Path(str(tile_dir) + '_4326')
    out_dir.mkdir(exist_ok=True, parents=True)

    tifs = list(tile_dir.glob('*.tif'))
    
    reference_profile = get_reference_tif_profile(tile_name)
    
    def reproj_partial(tif_path):
        return reproject_singe_band(tif_path, reference_profile, out_dir)
    
    out_tifs = list(map(reproj_partial, tqdm(tifs)))
    return out_tifs

In [12]:
reproject_tile_data(tiles[0])

100%|██████████| 4/4 [00:54<00:00, 14.01s/it]


[PosixPath('out/NH15_4326/distance.tif'),
 PosixPath('out/NH15_4326/ocean_mask.tif'),
 PosixPath('out/NH15_4326/segments.tif'),
 PosixPath('out/NH15_4326/water_mask.tif')]

In [13]:
list(map(reproject_tile_data, tqdm(tiles[1:])))

100%|██████████| 2/2 [01:49<00:00, 54.78s/it]


[[PosixPath('out/NH16_4326/distance.tif'),
  PosixPath('out/NH16_4326/ocean_mask.tif'),
  PosixPath('out/NH16_4326/segments.tif'),
  PosixPath('out/NH16_4326/water_mask.tif')],
 [PosixPath('out/NR08_4326/distance.tif'),
  PosixPath('out/NR08_4326/ocean_mask.tif'),
  PosixPath('out/NR08_4326/segments.tif'),
  PosixPath('out/NR08_4326/water_mask.tif')]]